In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    # Prompt the user to enter the API key if not set
    import getpass
    api_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")
    if not api_key:
        raise ValueError("OPENAI_API_KEY variable is not set in the environment variables or provided by user.")

In [4]:
from agents import Agent, Runner, TResponseInputItem

simple_agent = Agent(
    name="Malik Shb ka Agent",
    instructions="You are a helpful assistant. Reply very consisely.",
    model="gpt-4o-mini"
)

conversation: list[TResponseInputItem] = []

print("You are chating with Malik Shb ka Agent. Type 'exit' to quit.\n")
while True:
    user_input = input("You: ")
    print("You: "+ user_input)

    conversation.append({"role": "user", "content": user_input})
    if user_input.lower() == "exit":
        print("Exiting the chat. Goodbye!")
        break

    response = await Runner.run(simple_agent, conversation)
    print("Agent: " + response.final_output + "\n")
    conversation.append({"role": "assistant", "content": response.final_output})






You are chating with Malik Shb ka Agent. Type 'exit' to quit.

You: hey
Agent: Hello! How can I assist you today?

You: tell me about the president of Italy?
Agent: As of my last update, the President of Italy is Sergio Mattarella, in office since February 3, 2015. He serves as the head of state and represents national unity. His role is largely ceremonial, with powers including appointing the Prime Minister and overseeing the functioning of government.

You: capital of italy
Agent: The capital of Italy is Rome.

You: what was my second msg?
Agent: Your second message was asking about the president of Italy.

You: great man
Agent: Glad you think so! Would you like to know more about him?

You: tell me about yourself
Agent: I’m an AI assistant designed to help with information and tasks. How can I assist you today?

You: thank you please exit
Agent: You're welcome! If you need anything else, feel free to return. Have a great day!

You: exit
Exiting the chat. Goodbye!


In [5]:
print(conversation)

[{'role': 'user', 'content': 'hey'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': 'tell me about the president of Italy?'}, {'role': 'assistant', 'content': 'As of my last update, the President of Italy is Sergio Mattarella, in office since February 3, 2015. He serves as the head of state and represents national unity. His role is largely ceremonial, with powers including appointing the Prime Minister and overseeing the functioning of government.'}, {'role': 'user', 'content': 'capital of italy'}, {'role': 'assistant', 'content': 'The capital of Italy is Rome.'}, {'role': 'user', 'content': 'what was my second msg?'}, {'role': 'assistant', 'content': 'Your second message was asking about the president of Italy.'}, {'role': 'user', 'content': 'great man'}, {'role': 'assistant', 'content': 'Glad you think so! Would you like to know more about him?'}, {'role': 'user', 'content': 'tell me about yourself'}, {'role': 'assistant', 'conten